In [1]:
from chemloop.core.net_reactions import NetReaction
from chemloop.core.redox_materials import RedoxMaterialsSet
from chemloop.core.chemical_loops import ChemicalLoopTwoStep
from chemloop.core.networks import ChemicalLoopNetwork
from pymatgen.core import Composition
from rxn_network.enumerators.basic import BasicEnumerator

/home/jiaxin/miniconda3/envs/chemloop_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Define a chemical looping process
## 1. Net reaction

In [2]:
# chemical looping combustion: 2 O2 + CH4 -> CO2 + 2 H2O

net_rxn = NetReaction(oxidant=Composition("O2"),
                      reducing_agent=Composition("CH4"),
                      products=[Composition("CO2"), Composition("H2O")],
                      coefficients=[-2,-1, 1, 2]
                     )
net_rxn

NetReaction(oxidant=Comp: O2, reducing_agent=Comp: H4 C1, products=[Comp: C1 O2, Comp: H2 O1], coefficients=[-2, -1, 1, 2])

In [3]:
net_rxn.equation

'2 O2 + H4C -> CO2 + 2 H2O'

In [4]:
net_rxn.balanced

True

## 2. Redox materials

In [5]:
redox_materials = RedoxMaterialsSet(materials=[Composition("Fe2O3"), Composition("Fe3O4")])
redox_materials

## 3. Looping type

In [6]:
two_step = ChemicalLoopTwoStep(redox_pair=redox_materials, net_rxn=net_rxn)
two_step

In [7]:
print(two_step)

H4C + 12 Fe2O3 -> CO2 + 2 H2O + 8 Fe3O4
0.1667 O2 + 0.6667 Fe3O4 -> Fe2O3
Net rxn: 2 O2 + H4C -> CO2 + 2 H2O


## 4. Pathway finding in the subreactions

In [8]:
cln = ChemicalLoopNetwork(chemical_loop=two_step, temps=[600, 600],entries_from="api")

In [9]:
networks = cln.build_networks(enumerator=BasicEnumerator(), e_above_hull=0.00)

Source entries from: api


/home/jiaxin/miniconda3/envs/chemloop_env/lib/python3.10/site-packages/mp_api/client/mprester.py:146: UserWarning: mpcontribs-client not installed. Install the package to query MPContribs data, or construct pourbaix diagrams: 'pip install mpcontribs-client'
  warnings.warn(
Retrieving ThermoDoc documents: 100%|████████████████████████████████████████████| 926/926 [00:00<00:00, 7023373.42it/s]


Materials Project database version:  2022.10.28


2023-01-03 13:32:01,496	INFO worker.py:1538 -- Started a local Ray instance.
100%|█████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:04<00:00,  1.81it/s]


Building network for:  H4C + 12 Fe2O3 -> CO2 + 2 H2O + 8 Fe3O4


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 13.73it/s]


Building network for:  0.1667 O2 + 0.6667 Fe3O4 -> Fe2O3


100%|█████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 1621.33it/s]


In [10]:
computed_subrxns, pathways=cln.solve_pathways(verbose=False)

Solving pathways for:  H4C + 12 Fe2O3 -> CO2 + 2 H2O + 8 Fe3O4


PathwaySolver (Batch 1/1): 100%|██████████████████████████████████████████████████████████| 5/5 [00:17<00:00,  3.42s/it]


Solving pathways for:  0.1667 O2 + 0.6667 Fe3O4 -> Fe2O3


PathwaySolver (Batch 1/1): 100%|██████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 30.05it/s]


In [11]:
pathways[0].get_paths()

[6 Fe2O3 -> O2 + 4 Fe3O4 (dG = 0.108 eV/atom) 
 O2 + H4C -> C + 2 H2O (dG = -0.449 eV/atom) 
 O2 + C -> CO2 (dG = -1.365 eV/atom) 
 Average Cost: 0.296,
 6 Fe2O3 -> O2 + 4 Fe3O4 (dG = 0.108 eV/atom) 
 0.5 O2 + 0.5 H4C -> 0.5 CO2 + H2 (dG = -0.551 eV/atom) 
 O2 + H4C -> C + 2 H2O (dG = -0.449 eV/atom) 
 O2 + 2 H2 -> 2 H2O (dG = -0.563 eV/atom) 
 O2 + C -> CO2 (dG = -1.365 eV/atom) 
 Average Cost: 0.299,
 6 Fe2O3 -> O2 + 4 Fe3O4 (dG = 0.108 eV/atom) 
 H4C -> C + 2 H2 (dG = 0.047 eV/atom) 
 O2 + H4C -> C + 2 H2O (dG = -0.449 eV/atom) 
 O2 + 2 H2 -> 2 H2O (dG = -0.563 eV/atom) 
 O2 + C -> CO2 (dG = -1.365 eV/atom) 
 Average Cost: 0.302,
 6 Fe2O3 -> O2 + 4 Fe3O4 (dG = 0.108 eV/atom) 
 0.5 O2 + 0.5 H4C -> 0.5 CO2 + H2 (dG = -0.551 eV/atom) 
 O2 + 2 H2 -> 2 H2O (dG = -0.563 eV/atom) 
 Average Cost: 0.303,
 6 Fe2O3 -> O2 + 4 Fe3O4 (dG = 0.108 eV/atom) 
 2 Fe2O3 -> O2 + 4 FeO (dG = 0.413 eV/atom) 
 O2 + H4C -> C + 2 H2O (dG = -0.449 eV/atom) 
 FeO + Fe2O3 -> Fe3O4 (dG = -0.032 eV/atom) 
 O2 + C